<a href="https://colab.research.google.com/github/fernandodelaiglesia/cajondesastre/blob/master/QECwDist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BitFlip QEC with distribution

In [1]:
!pip install projectq

     |████████████████████████████████| 194kB 2.9MB/s 
     |████████████████████████████████| 153kB 42.6MB/s 
  ERROR: Failed building wheel for projectq
  Running setup.py clean for projectq
Failed to build projectq
  Running setup.py install for projectq ... done


In [0]:
from projectq import MainEngine
from projectq.backends import Simulator
from projectq.ops import StatePreparation, X, H, Measure, All
from projectq.meta import Loop, Control, Compute, Uncompute

import math



In [17]:
# List of non error and error ancilla result
nonerror_list = [[0, 0, 0], [0, 1, 1], [1, 0, 0], [1, 1, 1]]
error_list = [[0, 0, 1], [0, 1, 0], [1, 0, 1], [1, 1, 0]]

# Quantum process
eng = MainEngine()

info_qbits = eng.allocate_qureg(3)

ancilla1 = eng.allocate_qureg(3)
ancilla2 = eng.allocate_qureg(3)

amplitude0 = math.sqrt(3)/2.
amplitude1 = 1./2.
StatePreparation([amplitude0, amplitude1]) | info_qbits[0]

# Prepare the protected state
# Code: 3 redundant qubit

with Control(eng, info_qbits[0]):
  All(X) | info_qbits[1:]

# Prepare the entagled ancillas

H | ancilla1[0]
with Control(eng, ancilla1[0]):
  All(X) | ancilla1[1:]
  
H | ancilla2[0]
with Control(eng, ancilla2[0]):
  All(X) | ancilla2[1:]

# Distribute
# Nothing to do, this is local simulation

# A: info_qbits[0], ancilla1[1]
# B: info_qbits[1], ancilla1[2], ancilla2[1]
# C: info_qbits[2], ancilla2[2]

# Let's introduce some (bit flip) error

#X | info_qbits[2]

# QEC

# A
with Control(eng, info_qbits[0]):
  X | ancilla1[1]
  
# B
with Control(eng, info_qbits[1]):
  X | ancilla1[2]
  X | ancilla2[1]
  
# C
with Control(eng, info_qbits[2]):
  X | ancilla2[2]
  
# Measure the ancillas to check for errors

H | ancilla1[0]
H | ancilla2[0]

All(Measure) | ancilla1
All(Measure) | ancilla2

m_ancilla1 = [int(q) for q in ancilla1]
m_ancilla2 = [int(q) for q in ancilla2]

print("Ancilla measurement: 1: ",m_ancilla1, " , 2: ",m_ancilla2)

eng.flush()

prob000 = eng.backend.get_probability('000', info_qbits)
prob111 = eng.backend.get_probability('111', info_qbits)

print("Info qubits probability: |000>",prob000, " , |111>: ",prob111, " . It should be 0.75 and 0.25")

# Error detection and error recovery
if (m_ancilla1 in nonerror_list) and (m_ancilla2 in nonerror_list):
  print("\nNo error or evesdrop happens, cool!, No error recovery required\n")
elif (m_ancilla1 in error_list) and (m_ancilla2 in nonerror_list):
  print("\nError or Eve happens :-(  : m_ancilla1 = ", m_ancilla1, ", m_ancilla2 = ", m_ancilla2,"\n")
  X | info_qbits[0]
  print ("Error corrected with X0!\n")
elif (m_ancilla1 in error_list) and (m_ancilla2 in error_list):
  print("\nError or Eve happens :-(  : m_ancilla1 = ", m_ancilla1, ", m_ancilla2 = ", m_ancilla2,"\n")
  X | info_qbits[1]
  print ("Error corrected with X1!\n")
elif (m_ancilla1 in nonerror_list) and (m_ancilla2 in error_list):
  print("\nError or Eve happens :-(  : m_ancilla1 = ", m_ancilla1, ", m_ancilla2 = ", m_ancilla2,"\n")
  X | info_qbits[2]
  print ("Error corrected with X2!\n")


eng.flush()

prob000 = eng.backend.get_probability('000', info_qbits)
prob111 = eng.backend.get_probability('111', info_qbits)

print("Info qubits probability AFTER error recovery: |000>",prob000, " , |111>: ",prob111, " . It should be 0.75 and 0.25")

All(Measure) | info_qbits

eng.flush()



Ancilla measurement: 1:  [0, 1, 1]  , 2:  [0, 1, 1]
Info qubits probability: |000> 0.7500000000002589  , |111>:  0.24999999999974126  . It should be 0.75 and 0.25

No error or evesdrop happens, cool!, No error recovery required

Info qubits probability AFTER error recovery: |000> 0.7500000000002589  , |111>:  0.24999999999974126  . It should be 0.75 and 0.25
